# 🛣️ RoadScan AI — Colab Training
Run each cell top to bottom. Only edit the CONFIG cell.

## Cell 1 · Install

In [1]:
!pip install -q torch torchvision pyyaml scikit-learn

## Cell 2 · Imports

In [ ]:
import os, time, json
from pathlib import Path
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms, models
from torchvision.models import EfficientNet_V2_S_Weights
from PIL import Image

print("PyTorch:", torch.__version__)
print("CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

PyTorch: 2.10.0+cpu
CUDA: False


## Cell 3 · Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## ⚙️ Cell 4 · CONFIG — Edit this cell

In [4]:
DATA_DIR   = "/content/drive/MyDrive/maddata-hackathon-2026/datasets/final"
OUTPUT_DIR = "/content/drive/MyDrive/maddata-hackathon-2026/checkpoints"

EPOCHS     = 20
BATCH_SIZE = 32   # lower to 16 if out-of-memory
LR         = 1e-3
IMAGE_SIZE = 224

os.makedirs(OUTPUT_DIR, exist_ok=True)
print("Data:", DATA_DIR)
print("Output:", OUTPUT_DIR)

Data: /content/drive/MyDrive/maddata-hackathon-2026/datasets/final
Output: /content/drive/MyDrive/maddata-hackathon-2026/checkpoints


## Cell 5 · Load dataset.yaml

In [5]:
with open(f"{DATA_DIR}/dataset.yaml") as f:
    cfg = yaml.safe_load(f)

CLASS_NAMES  = cfg["names"]
NUM_CLASSES  = len(CLASS_NAMES)
class_to_idx = {name: i for i, name in enumerate(CLASS_NAMES)}

TRAIN_IMG_DIR = Path(DATA_DIR) / "train" / "images"
VAL_IMG_DIR   = Path(DATA_DIR) / "val"   / "images"

print("Classes:", CLASS_NAMES)
print("Train dir:", TRAIN_IMG_DIR, "| exists:", TRAIN_IMG_DIR.exists())
print("Val dir:  ", VAL_IMG_DIR,   "| exists:", VAL_IMG_DIR.exists())

Classes: ['pothole', 'garbage_overflow', 'broken_streetlight', 'water_leakage', 'broken_sidewalk', 'sinkhole']
Train dir: /content/drive/MyDrive/maddata-hackathon-2026/datasets/final/train/images | exists: True
Val dir:   /content/drive/MyDrive/maddata-hackathon-2026/datasets/final/val/images | exists: True


## Cell 6 · Device

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CPU only — training will be slow")

CPU only — training will be slow


## Cell 7 · Dataset

Class is matched from the filename prefix — longest match wins, so `broken_streetlight_abc.jpg` → `broken_streetlight`, not `broken_sidewalk`.

In [7]:
VALID_EXTS    = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
sorted_classes = sorted(CLASS_NAMES, key=len, reverse=True)  # longest first

def match_class(stem):
    for cls in sorted_classes:
        if stem == cls or stem.startswith(cls + "_"):
            return cls
    return None

class RoadDataset(Dataset):
    def __init__(self, images_dir, transform):
        self.transform = transform
        self.samples   = []
        skipped = 0
        for p in sorted(Path(images_dir).iterdir()):
            if p.suffix.lower() not in VALID_EXTS:
                continue
            cls = match_class(p.stem)
            if cls is not None:
                self.samples.append((p, class_to_idx[cls]))
            else:
                skipped += 1
        print(f"  {len(self.samples)} images loaded, {skipped} skipped")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        return self.transform(Image.open(path).convert("RGB")), label

    @property
    def targets(self):
        return [lbl for _, lbl in self.samples]

## Cell 8 · Transforms & DataLoaders

In [8]:
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

train_tf = transforms.Compose([
    transforms.Resize((IMAGE_SIZE + 24, IMAGE_SIZE + 24)),
    transforms.RandomCrop(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
val_tf = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

print("Loading train dataset...")
train_ds = RoadDataset(TRAIN_IMG_DIR, train_tf)
print("Loading val dataset...")
val_ds   = RoadDataset(VAL_IMG_DIR, val_tf)

# Weighted sampler to balance class imbalance
targets     = torch.tensor(train_ds.targets)
class_count = torch.bincount(targets, minlength=NUM_CLASSES).float()
class_count[class_count == 0] = 1
weights     = 1.0 / class_count[targets]
sampler     = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

NUM_WORKERS  = 2 if device.type == "cuda" else 0
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler,  num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print("\nClass distribution (train):")
for i, cls in enumerate(CLASS_NAMES):
    print(f"  [{i}] {cls:<25s}: {int(class_count[i])} images")

Loading train dataset...
  11356 images loaded, 0 skipped
Loading val dataset...
  3216 images loaded, 0 skipped

Class distribution (train):
  [0] pothole                  : 1303 images
  [1] garbage_overflow         : 1961 images
  [2] broken_streetlight       : 6927 images
  [3] water_leakage            : 61 images
  [4] broken_sidewalk          : 183 images
  [5] sinkhole                 : 921 images


## Cell 9 · Build Model

In [9]:
model = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)

# Freeze backbone — train head only during warm-up
for param in model.features.parameters():
    param.requires_grad = False

# Replace classifier for our classes
in_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(256, NUM_CLASSES),
)
model = model.to(device)
print("Model ready — trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Model ready — trainable params: 329478


## Cell 10 · Train

- **Epochs 1–3 (warm-up):** only the classifier head trains, backbone frozen.
- **Epoch 4+:** last backbone blocks unfreeze for full fine-tuning.
- Stops early if val accuracy doesn't improve for 5 epochs.

In [ ]:
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
scaler    = torch.amp.GradScaler("cuda") if device.type == "cuda" else None
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

WARMUP   = 3
PATIENCE = 5
best_acc = 0.0
no_imp   = 0
history  = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
ckpt_path = Path(OUTPUT_DIR) / "best_roadscan.pt"

for epoch in range(1, EPOCHS + 1):
    t0 = time.time()

    # Unfreeze backbone after warm-up
    if epoch == WARMUP + 1:
        print("\n[Phase 2] Unfreezing backbone blocks 6+")
        for i, layer in enumerate(model.features):
            if i >= 6:
                for p in layer.parameters():
                    p.requires_grad = True
        optimizer = optim.AdamW([
            {"params": model.features.parameters(),   "lr": LR * 0.1},
            {"params": model.classifier.parameters(), "lr": LR},
        ], weight_decay=1e-4)
        scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS - epoch)

    # ── Train ──────────────────────────────────────────────────────────────
    model.train()
    tr_loss, tr_correct, tr_total = 0.0, 0, 0

    train_bar = tqdm(train_loader, desc=f"Epoch {epoch:02d}/{EPOCHS} [Train]", leave=False)
    for imgs, labels in train_bar:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad(set_to_none=True)

        if device.type == "cuda":
            with torch.amp.autocast("cuda"):
                out  = model(imgs)
                loss = criterion(out, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            out  = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()

        tr_loss    += loss.item() * imgs.size(0)
        tr_correct += (out.argmax(1) == labels).sum().item()
        tr_total   += imgs.size(0)

        train_bar.set_postfix(loss=f"{tr_loss/tr_total:.4f}", acc=f"{tr_correct/tr_total:.3f}")

    # ── Validate ───────────────────────────────────────────────────────────
    model.eval()
    vl_loss, vl_correct, vl_total = 0.0, 0, 0
    all_preds, all_labels = [], []

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch:02d}/{EPOCHS} [Val]  ", leave=False)
    with torch.no_grad():
        for imgs, labels in val_bar:
            imgs, labels = imgs.to(device), labels.to(device)
            out   = model(imgs)
            loss  = criterion(out, labels)
            preds = out.argmax(1)

            vl_loss    += loss.item() * imgs.size(0)
            vl_correct += (preds == labels).sum().item()
            vl_total   += imgs.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            val_bar.set_postfix(loss=f"{vl_loss/vl_total:.4f}", acc=f"{vl_correct/vl_total:.3f}")

    scheduler.step()

    tr_acc = tr_correct / tr_total
    vl_acc = vl_correct / vl_total
    history["train_loss"].append(tr_loss / tr_total)
    history["train_acc"].append(tr_acc)
    history["val_loss"].append(vl_loss / vl_total)
    history["val_acc"].append(vl_acc)

    print(f"Epoch {epoch:02d}/{EPOCHS} | Train loss {tr_loss/tr_total:.4f} acc {tr_acc:.3f} | Val loss {vl_loss/vl_total:.4f} acc {vl_acc:.3f} | {time.time()-t0:.1f}s")

    if vl_acc > best_acc:
        best_acc = vl_acc
        no_imp   = 0
        torch.save({"model_state": model.state_dict(), "classes": CLASS_NAMES,
                    "class_to_idx": class_to_idx, "image_size": IMAGE_SIZE}, ckpt_path)
        print(f"  💾 Saved best model (val_acc={vl_acc:.4f})")
    else:
        no_imp += 1
        if no_imp >= PATIENCE:
            print(f"\n⏹️  Early stopping — no improvement for {PATIENCE} epochs")
            break

with open(Path(OUTPUT_DIR) / "history.json", "w") as f:
    json.dump(history, f, indent=2)

print(f"\n✅ Done. Best val acc: {best_acc:.4f}")
print(f"   Checkpoint: {ckpt_path}")

Epoch 01/20 [Train]:   0%|          | 0/355 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
Epoch 01/20 [Train]:   1%|          | 3/355 [01:04<2:03:54, 21.12s/it, acc=0.240, loss=1.7563]

## Cell 11 · Per-Class Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(all_labels, all_preds, target_names=CLASS_NAMES, digits=3))

## Cell 12 · Plot Training Curves

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(history["train_loss"], label="Train"); ax1.plot(history["val_loss"], label="Val")
ax1.set_title("Loss"); ax1.set_xlabel("Epoch"); ax1.legend(); ax1.grid(True)
ax2.plot(history["train_acc"], label="Train"); ax2.plot(history["val_acc"], label="Val")
ax2.set_title("Accuracy"); ax2.set_xlabel("Epoch"); ax2.legend(); ax2.grid(True)
plt.tight_layout()
plt.savefig(Path(OUTPUT_DIR) / "training_curves.png", dpi=120)
plt.show()